#### RF with RFE and hyperparam tuning

In [25]:
train_df = pd.read_csv("checkpoint_train.csv")
test_df = pd.read_csv("checkpoint_test.csv")    

In [26]:
train_df.drop(columns = text_columns, inplace = True)
test_df.drop(columns = text_columns, inplace = True)

In [27]:
X_train = train_df.drop(labels = 'rating', axis = 1)
y_train = train_df['rating']

X_test = test_df

In [28]:
# Create Random Forest Regression model
model = RandomForestRegressor()

# Create RFE object
rfe = RFE(estimator = model)

In [29]:

# Create parameter grid for RFE hyperparameters
rfe_param_grid = {
    'n_features_to_select': [5, 10, 15],
    'estimator__n_estimators': [50, 100, 200],
    'estimator__max_depth': [3, 5, None]
}


In [30]:

# Perform grid search with cross-validation for RFE hyperparameters
rfe_grid_search = GridSearchCV(estimator = rfe, param_grid = rfe_param_grid, cv = 5, n_jobs = -1)
rfe_grid_search.fit(X_train, y_train)


KeyboardInterrupt: 

In [ ]:
# Get best RFE parameters and transform training and testing data
rfe_best_params = rfe_grid_search.best_params_
rfe.set_params(**rfe_best_params)
rfe.fit(X_train, y_train)
X_train_rfe = rfe.transform(X_train)
X_test_rfe = rfe.transform(X_test)

In [ ]:
# Create parameter grid for Random Forest Regression hyperparameters
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, None],
    'max_features': ['sqrt', 'log2', None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:

# Perform grid search with cross-validation for Random Forest Regression hyperparameters
rf_grid_search = GridSearchCV(estimator = model, param_grid = rf_param_grid, cv = 5, n_jobs = -1)
rf_grid_search.fit(X_train_rfe, y_train)


In [ ]:
# Get best Random Forest Regression parameters and train final model
rf_best_params = rf_grid_search.best_params_
model.set_params(**rf_best_params)
model.fit(X_train_rfe, y_train)


In [ ]:
test_df['rating'] = (model.predict(X_test_rfe))
test_df['movieId'] = test_df['movieId'].astype(int).astype(str)
test_df['userId'] = test_df['userId'].astype(int).astype(str)
test_df['userId_movieId'] = test_df['userId'] + '_' + test_df['movieId']
cols = ['userId_movieId', 'rating']
test_df = test_df[cols]

In [ ]:
test_df.to_csv('submission_rf_tuned_with_feature_selection.csv', index = False)